# IU Applied Data Science (INFO_I-590)
### Instructor: Joanne Luciano ; TAs: Jeremy Yang and Kaicheng Yang
# ADS Twitter API Tutorial (Part 2): Feature vectorization and other text processing details
### Based on original tutorial and code by former TA JT Wolohan.
### *NOTE: This tutorial code does NOT fulfill all requrements of assignment.  It provides a good start!*
### References:

*   https://apps.twitter.com/
*   https://github.com/geduldig/TwitterAPI
*   https://media.readthedocs.org/pdf/twitterapi/latest/twitterapi.pdf

### Import packages

In [30]:
import os, json
import pandas
import numpy
import sklearn
import sklearn.feature_extraction
import sklearn.model_selection 
import sklearn.metrics 
import sklearn.naive_bayes
import sklearn.svm
import sklearn.neighbors
import sklearn.neural_network
!pip install TwitterAPI
from TwitterAPI import TwitterAPI, TwitterOAuth

### Twitter API Credentials (IN ACCORDANCE WITH TWITTER EULA, DO NOT COPY, USE ONLY YOUR CREDENTIALS.)

In [0]:
consumer_key_b64 = 'cmJqbG1US1JwdU04ZXhJQlBPWnlvbFZRSw=='
consumer_secret_b64  = 'enNrMjdSNGlLbElOUzN4YWk3WHFzQnAxVkdyaU1xU0FFYzlDSGd5SUhselZvRjZ1dmw='
access_token_key_b64 = 'NjY4MzUxMDMtN3dKUFpPUHhjZERzbkY0RkViQ0E3RlFnYjhENEFQYjk1TDhaaWZrZUo='
access_token_secret_b64 = 'S0dDczUyZnZvRjhRRDZPcktEU3JxRlFhSWZOZkRnTng2aWJlWUVrZWt0Y3F0'

### SAVE YOUR APPLICATION CREDENTIALS

In [0]:
from base64 import b64decode

consumer_key = str(b64decode(consumer_key_b64), 'utf8')
consumer_secret = str(b64decode(consumer_secret_b64), 'utf8')
access_token_key = str(b64decode(access_token_key_b64), 'utf8')
access_token_secret = str(b64decode(access_token_secret_b64), 'utf8')

# Credentials file format:
# consumer_key=YOUR_CONSUMER_KEY
# consumer_secret=YOUR_CONSUMER_SECRET
# access_token_key=YOUR_ACCESS_TOKEN
# access_token_secret=YOUR_ACCESS_TOKEN_SECRET

fpath = os.environ['HOME']+'/.twitterapi_credentials'
fout = open(fpath, "w")
fout.write("consumer_key=%s\n"%consumer_key)
fout.write("consumer_secret=%s\n"%consumer_secret)
fout.write("access_token_key=%s\n"%access_token_key)
fout.write("access_token_secret=%s\n"%access_token_secret)
fout.close()

### Connect and authenticate with Twitter REST API.

In [0]:
o = TwitterOAuth.read_file(os.environ['HOME']+'/.twitterapi_credentials')
twapi = TwitterAPI(o.consumer_key, o.consumer_secret, o.access_token_key, o.access_token_secret)

### Convenience function

In [0]:
def searchTwitter(q, api, feed="search/tweets", n=100):
  return [t for t in api.request(feed, {'q':q,'count':n})]

### Get JSON from Twitter

In [0]:
cat_tweets = searchTwitter('#cats', twapi, n=100)
dog_tweets = searchTwitter('#dogs', twapi, n=100)

### Convert the json returned by Twitter into a dataframe

In [0]:
cat_df = pandas.read_json(json.dumps(cat_tweets))
dog_df = pandas.read_json(json.dumps(dog_tweets))

### Inspect data frames.


In [37]:
pandas.set_option('display.max_columns', None) 
cat_df.head()
dog_df.head()

,contributors,coordinates,created_at,entities,extended_entities,favorite_count,favorited,geo,id,id_str,in_reply_to_screen_name,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,is_quote_status,lang,metadata,place,possibly_sensitive,quoted_status,quoted_status_id,quoted_status_id_str,retweet_count,retweeted,retweeted_status,source,text,truncated,user
0,NaN,None,2018-03-01 15:45:41,"{'hashtags': [{'text': 'throwbackthursday', 'indices': [18, 36]}, {'text': 'dartmouth', 'indices...",NaN,0,False,None,969237246439231488,969237246439231488,NaN,NaN,NaN,NaN,NaN,False,en,"{'iso_language_code': 'en', 'result_type': 'recent'}",None,NaN,NaN,NaN,NaN,1,False,"{'created_at': 'Thu Mar 01 15:39:58 +0000 2018', 'id': 969235806761246722, 'id_str': '9692358067...","<a href=""http://twitter.com"" rel=""nofollow"">Twitter Web Client</a>","RT @bideawhile: . #throwbackthursday - This is where Bide Awhile began, almost 50 years ago on E...",False,"{'id': 26558733, 'id_str': '26558733', 'name': 'Halifax Chamber', 'screen_name': 'halifaxchamber..."
1,NaN,None,2018-03-01 15:45:40,"{'hashtags': [{'text': 'Dogs', 'indices': [99, 104]}, {'text': 'Maidstone', 'indices': [105, 115...",NaN,0,False,None,969237239937953792,969237239937953792,NaN,NaN,NaN,NaN,NaN,False,en,"{'iso_language_code': 'en', 'result_type': 'recent'}",None,0.0,NaN,NaN,NaN,2,False,"{'created_at': 'Thu Mar 01 15:17:46 +0000 2018', 'id': 969230221646946305, 'id_str': '9692302216...","<a href=""http://twitter.com/download/android"" rel=""nofollow"">Twitter for Android</a>",RT @dogwalkingmaids: Dog Walking in Maidstone from Dog Walking Maidstone - https://t.co/ZuWCiY9k...,False,"{'id': 3057634784, 'id_str': '3057634784', 'name': 'hirotoki 1639w1095', 'screen_name': 'hirotok..."
2,NaN,None,2018-03-01 15:45:32,"{'hashtags': [{'text': 'lévriers', 'indices': [0, 9]}, {'text': 'dogs', 'indices': [10, 15]}, {'...","{'media': [{'id': 969236720679030785, 'id_str': '969236720679030785', 'indices': [106, 129], 'me...",0,False,None,969237206169767937,969237206169767936,NaN,NaN,NaN,NaN,NaN,False,fr,"{'iso_language_code': 'fr', 'result_type': 'recent'}",None,0.0,NaN,NaN,NaN,0,False,NaN,"<a href=""http://twitter.com"" rel=""nofollow"">Twitter Web Client</a>",#lévriers #dogs #Nikon #D3200 \nMon trépied :https://t.co/huT2XW5Us5\nMon REFLEX :https://t.co...,False,"{'id': 740617701756960768, 'id_str': '740617701756960768', 'name': 'Sodropsss', 'screen_name': '..."
3,NaN,None,2018-03-01 15:45:26,"{'hashtags': [{'text': 'AmorAnimal', 'indices': [17, 28]}, {'text': 'alma', 'indices': [29, 34]}...","{'media': [{'id': 969186457607921664, 'id_str': '969186457607921664', 'indices': [64, 87], 'medi...",0,False,None,969237181675069441,969237181675069440,NaN,NaN,NaN,NaN,NaN,False,und,"{'iso_language_code': 'und', 'result_type': 'recent'}",None,0.0,NaN,NaN,NaN,49,False,"{'created_at': 'Thu Mar 01 12:23:53 +0000 2018', 'id': 969186460921483264, 'id_str': '9691864609...","<a href=""http://twitter.com"" rel=""nofollow"">Twitter Web Client</a>",RT @elcampito: ❤ #AmorAnimal #alma #perros #rescate #dogs #love https://t.co/etDqQIADv5,False,"{'id': 228494025, 'id_str': '228494025', 'name': 'Nahuel González', 'screen_name': 'zzahuel', 'l..."
4,NaN,None,2018-03-01 15:45:18,"{'hashtags': [{'text': 'goodboy', 'indices': [34, 42]}, {'text': 'dogs', 'indices': [43, 48]}], ...","{'media': [{'id': 969065547575975936, 'id_str': '969065547575975936', 'indices': [53, 76], 'medi...",0,False,None,969237150653919232,969237150653919232,NaN,NaN,NaN,NaN,NaN,False,en,"{'iso_language_code': 'en', 'result_type': 'recent'}",None,0.0,NaN,NaN,NaN,201,False,"{'created_at': 'Thu Mar 01 04:23:53 +0000 2018', 'id': 969065665142210560, 'id_str': '9690656651...","<a href=""https://mobile.twitter.com"" rel=""nofollow"">Twitter Lite</a>",RT @Cory_1077: Wait for it..... \n\n#goodboy #dogs 😂🐾💞 https://t.co/bSfhfCiuxW,False,"{'id': 761929682048602112, 'id_str': '761929682048602112', 'name': 'Pamela Crawford', 'screen_na...

In [38]:
cat_df.loc[1]

contributors                                                                                                                 NaN
coordinates                                                                                                                 None
created_at                                                                                                   2018-03-01 15:45:15
entities                     {'hashtags': [{'text': 'DidYouKnjow', 'indices': [0, 12]}, {'text': 'pethacks', 'indices': [106,...
extended_entities                                                                                                            NaN
favorite_count                                                                                                                 0
favorited                                                                                                                  False
geo                                                                                              

In [39]:
pandas.set_option('display.max_colwidth', 100)
cat_df.loc[1:10]['text']

1     #DidYouKnjow The word 'tabby' is said to derive from the name Attabiyah, a neighborhood in Baghd...
2     RT @Loki_the_lilac: The face you make when you're hooman asks if you want to go outside In the S...
3     Můj malý, modrý kamarád...\n\n#kocky #ruskamodra #russianblue #cats #kitten https://t.co/7WNGoiynI9
4     Pope Benedict loves cats...and they seem quite fond of him too.\n#PopeBenedictXVI #cats #CatsOfT...
5                                                      Kay's visiting :3\n\n#cats https://t.co/dJ74PAh4yM
6     Oh, no! Marsik needs our help. Must find safe refuge today.  Please join efforts to save Marsik....
7                       Mood: \n#ПисьмоНьютаСосетВИнсту #BlackPanther #cats #mood https://t.co/YORoFO9TLV
8     RT @CamrynZeePhoto: https://t.co/VsyGhsmt6M\n\nCat in Shadows\n\n#cats #cat #artwork #art #artis...
9     https://t.co/VsyGhsmt6M\n\nCat in Shadows\n\n#cats #cat #artwork #art #artist #digitalart #Photo...
10    RT @MaryAgnetha: С первым днём весны и с

### Get text only and replace hashtags with blanks
If you want to use the normalizer, import it above and pass x.replace() to the noramlizer function

In [0]:
cat_txt = [x.replace('#cats',"BLAH") for x in cat_df['text']]
dog_txt = [x.replace('#dogs',"BLAH") for x in dog_df['text']]

In [41]:
cat_txt[1:10]

["#DidYouKnjow The word 'tabby' is said to derive from the name Attabiyah, a neighborhood in Baghdad, Iran. #pethacks BLAH",
 "RT @Loki_the_lilac: The face you make when you're hooman asks if you want to go outside In the Snow 💙🌧😹 #SnowDay BLAH #cat #CatsOfTwitter…",
 'Můj malý, modrý kamarád...\n\n#kocky #ruskamodra #russianblue BLAH #kitten https://t.co/7WNGoiynI9',
 'Pope Benedict loves cats...and they seem quite fond of him too.\n#PopeBenedictXVI BLAH #CatsOfTwitter… https://t.co/a12G8XI8zM',
 "Kay's visiting :3\n\nBLAH https://t.co/dJ74PAh4yM",
 'Oh, no! Marsik needs our help. Must find safe refuge today.  Please join efforts to save Marsik. BLAH #Philly https://t.co/c8GdV6r6X2',
 'Mood: \n#ПисьмоНьютаСосетВИнсту #BlackPanther BLAH #mood https://t.co/YORoFO9TLV',
 'RT @CamrynZeePhoto: https://t.co/VsyGhsmt6M\n\nCat in Shadows\n\nBLAH #cat #artwork #art #artist #digitalart #Photography #500pxrtg #fineart #…',
 'https://t.co/VsyGhsmt6M\n\nCat in Shadows\n\nBLAH #cat #artwork #art #a

### Vectorization, Feature Selection and Extraction
* Train Scikit-Learn CountVectorizer on corpus of tweets, to define terms. 
* Extract term-counts and generate term-document matrices.

In [0]:
vectorizer = sklearn.feature_extraction.text.CountVectorizer(cat_txt+dog_txt, analyzer='word', stop_words=None, min_df=5)
vectorizer.fit(cat_txt+dog_txt)
cat_tdm = vectorizer.transform(cat_txt).toarray()
dog_tdm = vectorizer.transform(dog_txt).toarray()

In [43]:
vectorizer.get_feature_names()

['03',
 '18',
 'all',
 'amp',
 'an',
 'and',
 'animallovers',
 'animals',
 'are',
 'art',
 'blah',
 'can',
 'cat',
 'catlovers',
 'catoftheday',
 'cats',
 'catsoftwitter',
 'choco_sandy1',
 'co',
 'craic',
 'cute',
 'day',
 'do',
 'dog',
 'dogs',
 'feb',
 'for',
 'from',
 'fryld',
 'garrethdunleavy',
 'great',
 'helpinglostpets',
 'home',
 'https',
 'in',
 'is',
 'it',
 'just',
 'kitten',
 'kittens',
 'kitty',
 'ln5vw08gaj',
 'lost',
 'love',
 'markgkirshner',
 'my',
 'now',
 'oadele53',
 'of',
 'old',
 'on',
 'or',
 'out',
 'patdefranchis',
 'pets',
 'rescue',
 'rt',
 'saravastiares',
 'seen',
 'sneachta',
 'snow',
 'so',
 'stormemma',
 'that',
 'the',
 'this',
 'to',
 'today',
 'usa',
 've',
 'what',
 'when',
 'where',
 'you',
 'your']

In [44]:
cat_tdm[1:5]

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0,
        1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 2, 0,
        1, 0, 0, 0, 0, 1, 0, 3, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
    

### Combine matricies, adding class labels for training and testing.

In [0]:
catdog_tdm = numpy.concatenate((cat_tdm,dog_tdm),axis=0)
Ycat = numpy.array([0 for i in range(len(cat_txt))])
Ydog = numpy.array([1 for i in range(len(dog_txt))])
Y = numpy.concatenate((Ycat,Ydog),axis=0)

### Redo with stopwords list from NLTK
NLTK = Natural Language Toolkit (https://www.nltk.org/)

In [46]:
import nltk
import nltk.corpus
nltk.download('stopwords')
stopwds = list(nltk.corpus.stopwords.words('english'))
stopwds

[nltk_data] Downloading package stopwords to /content/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [47]:
vectorizer = sklearn.feature_extraction.text.CountVectorizer(cat_txt+dog_txt, analyzer='word', stop_words=stopwds, min_df=5)
vectorizer.fit(cat_txt+dog_txt)
cat_tdm = vectorizer.transform(cat_txt).toarray()
dog_tdm = vectorizer.transform(dog_txt).toarray()
vectorizer.get_feature_names()

['03',
 '18',
 'amp',
 'animallovers',
 'animals',
 'art',
 'blah',
 'cat',
 'catlovers',
 'catoftheday',
 'cats',
 'catsoftwitter',
 'choco_sandy1',
 'co',
 'craic',
 'cute',
 'day',
 'dog',
 'dogs',
 'feb',
 'fryld',
 'garrethdunleavy',
 'great',
 'helpinglostpets',
 'home',
 'https',
 'kitten',
 'kittens',
 'kitty',
 'ln5vw08gaj',
 'lost',
 'love',
 'markgkirshner',
 'oadele53',
 'old',
 'patdefranchis',
 'pets',
 'rescue',
 'rt',
 'saravastiares',
 'seen',
 'sneachta',
 'snow',
 'stormemma',
 'today',
 'usa']